In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk.corpus import wordnet

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import numpy as np
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [4]:
df=pd.read_csv('reddit_opinion_PSE_ISR.csv', encoding='latin1')

C:\Users\candi\AppData\Local\Temp\ipykernel_19584\151289139.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('reddit_opinion_PSE_ISR.csv', encoding='latin1')


In [5]:
df.columns

Index(['comment_id', 'score', 'self_text', 'subreddit', 'created_time',
       'post_id', 'author_name', 'controversiality', 'ups', 'downs',
       'user_is_verified', 'user_account_created_time', 'user_awardee_karma',
       'user_awarder_karma', 'user_link_karma', 'user_comment_karma',
       'user_total_karma', 'post_score', 'post_self_text', 'post_title',
       'post_upvote_ratio', 'post_thumbs_ups', 'post_total_awards_received',
       'post_created_time'],
      dtype='object')

In [6]:
df.dtypes

comment_id                     object
score                           int64
self_text                      object
subreddit                      object
created_time                   object
post_id                        object
author_name                    object
controversiality                int64
ups                             int64
downs                           int64
user_is_verified               object
user_account_created_time      object
user_awardee_karma            float64
user_awarder_karma            float64
user_link_karma               float64
user_comment_karma            float64
user_total_karma              float64
post_score                      int64
post_self_text                 object
post_title                     object
post_upvote_ratio             float64
post_thumbs_ups                 int64
post_total_awards_received      int64
post_created_time              object
dtype: object

In [7]:
df.shape

(2496456, 24)

In [8]:
df.head(2)

,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,...,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
0,miryacx,1,Hahahahah. Okay 20 âArab countriesââ¦ dud...,IsraelPalestine,2025-03-20 11:06:27,1jfl54o,ButterscotchThis5023,0,1,0,...,1.0,-17.0,-16.0,9,This is more of a rant. But yes Iâm an Arab(...,Iâm an Arab Jew living in America,0.74,9,0,2025-03-20 09:43:06
1,miry94o,1,You forgot the blockade lift was also taken aw...,IsraelPalestine,2025-03-20 11:06:09,1jfjx8s,Evening_Music9033,0,1,0,...,1.0,2233.0,2234.0,0,"After more than a year of conflict, a ceasefir...",Israel and their blatant ceasefire violations,0.37,0,0,2025-03-20 08:06:53


In [9]:
# Convert 'created_time' and 'post_created_time' to datetime format
df['created_time'] = pd.to_datetime(df['created_time'], errors='coerce')
df['post_created_time'] = pd.to_datetime(df['post_created_time'], errors='coerce')

In [10]:
# Convert text columns to string
df['self_text'] = df['self_text'].astype(str)
df['post_self_text'] = df['post_self_text'].astype(str)
df['post_title'] = df['post_title'].astype(str)

In [11]:
df = df[df['self_text'].str.strip() != '']
df = df[df['self_text'].str.len() > 20] 
df = df[df['post_self_text'].str.strip() != '']
df = df[df['post_self_text'].str.len() > 20] 

In [12]:
start_date = pd.to_datetime("2023-01-01")
end_date = pd.to_datetime("2024-07-31")
data = df[(df['created_time'] >= start_date) & (df['created_time'] <= end_date)]
data.reset_index(drop=True, inplace=True)
data.shape

(709940, 24)

In [13]:
content = data[[ 'comment_id', 'self_text', 'created_time','score','subreddit']]

In [14]:
content.head(5)

,comment_id,self_text,created_time,score,subreddit
0,lfqhets,Have you ever read Hamas mission statement fro...,2024-07-31 00:00:00,1,IsraelPalestine
1,lfqhbzr,You canât achieve zero casualties. But you ...,2024-07-30 23:59:31,-1,IsraelPalestine
2,lfqhb8g,You believe helping terrorists will serve your...,2024-07-30 23:59:23,6,IsraelPalestine
3,lfqh7c9,"They'll commit the worst genocide ever, then t...",2024-07-30 23:58:42,1,IsraelPalestine
4,lfqh6tr,"25,000 is a long shot. It took about 3000 Hama...",2024-07-30 23:58:37,1,IsraelPalestine


In [15]:
content['clean_text']=content['self_text'].str.lower()

C:\Users\candi\AppData\Local\Temp\ipykernel_19584\2586475655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_text']=content['self_text'].str.lower()


In [16]:
stop=stopwords.words('english')
stop.extend(['http', 'https', 'www', 'amp', 'com', 'would', 'rt'])
content['clean_text']=content['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
content['clean_text']=content['clean_text'].apply(word_tokenize)

C:\Users\candi\AppData\Local\Temp\ipykernel_19584\4008826710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_text']=content['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
C:\Users\candi\AppData\Local\Temp\ipykernel_19584\4008826710.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_text']=content['clean_text'].apply(word_tokenize)


In [17]:
content.head()

,comment_id,self_text,created_time,score,subreddit,clean_text
0,lfqhets,Have you ever read Hamas mission statement fro...,2024-07-31 00:00:00,1,IsraelPalestine,"[ever, read, hamas, mission, statement, 1988, ..."
1,lfqhbzr,You canât achieve zero casualties. But you ...,2024-07-30 23:59:31,-1,IsraelPalestine,"[canât, achieve, zero, casualties, ., honest..."
2,lfqhb8g,You believe helping terrorists will serve your...,2024-07-30 23:59:23,6,IsraelPalestine,"[believe, helping, terrorists, serve, interest..."
3,lfqh7c9,"They'll commit the worst genocide ever, then t...",2024-07-30 23:58:42,1,IsraelPalestine,"[commit, worst, genocide, ever, ,, terrorists,..."
4,lfqh6tr,"25,000 is a long shot. It took about 3000 Hama...",2024-07-30 23:58:37,1,IsraelPalestine,"[25,000, long, shot, ., took, 3000, hamas, mil..."


In [18]:
content['clean_text']=content['clean_text'].apply(lambda x: [word for word in x if word.isalnum()])

C:\Users\candi\AppData\Local\Temp\ipykernel_19584\3370924601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_text']=content['clean_text'].apply(lambda x: [word for word in x if word.isalnum()])


In [19]:
content.head()

,comment_id,self_text,created_time,score,subreddit,clean_text
0,lfqhets,Have you ever read Hamas mission statement fro...,2024-07-31 00:00:00,1,IsraelPalestine,"[ever, read, hamas, mission, statement, 1988, ..."
1,lfqhbzr,You canât achieve zero casualties. But you ...,2024-07-30 23:59:31,-1,IsraelPalestine,"[achieve, zero, casualties, honest, say, sides..."
2,lfqhb8g,You believe helping terrorists will serve your...,2024-07-30 23:59:23,6,IsraelPalestine,"[believe, helping, terrorists, serve, interests]"
3,lfqh7c9,"They'll commit the worst genocide ever, then t...",2024-07-30 23:58:42,1,IsraelPalestine,"[commit, worst, genocide, ever, terrorists, ru..."
4,lfqh6tr,"25,000 is a long shot. It took about 3000 Hama...",2024-07-30 23:58:37,1,IsraelPalestine,"[long, shot, took, 3000, hamas, militants, kil..."


In [20]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'): #ADJECTIVE
        return wordnet.ADJ
    elif nltk_tag.startswith('V'): #VERB
        return wordnet.VERB
    elif nltk_tag.startswith('N'): #NOUN        
        return wordnet.NOUN
    elif nltk_tag.startswith('R'): #ADVERB
        return wordnet.ADV
    else:          
        return None

In [21]:
def lemmatize_sentence(sentence):
    # Tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    # Tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged) 
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # If no tag was found, then use the word as is
            lemmatized_sentence.append(word)
        else:        
            # Else use the tag to lemmatize the word
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [22]:
content['clean_text']=content['clean_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
content['clean_text']=content['clean_text'].apply(lambda x: lemmatize_sentence(' '.join(x)))

C:\Users\candi\AppData\Local\Temp\ipykernel_19584\3301199086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_text']=content['clean_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
C:\Users\candi\AppData\Local\Temp\ipykernel_19584\3301199086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_text']=content['clean_text'].apply(lambda x: lemmatize_sentence(' '.join(x)))


In [23]:
print(content['clean_text'])

0         ever read hamas mission statement 1988 recentl...
1         achieve zero casualty honest say side care kil...
2                     believe help terrorist serve interest
3         commit bad genocide ever terrorist run country...
4         long shot take 3000 hamas militant kill less h...
                                ...                        
709935                               try stay calm daughter
709936                 robbery take anything sound like hit
709937          robbery hit guy plan kill even move towards
709938    go home wife kid act normal like traumatized c...
709939    fuck politically correct anyone shoot anyone f...
Name: clean_text, Length: 709940, dtype: object


In [24]:
content.to_csv('clean_data.csv', index=False)

In [25]:
analyzer = SentimentIntensityAnalyzer()
scores = {
    "Compound": [],
    "Positive": [],
    "Negative": [],
    "Neutral": []
}
for comment in content['clean_text']:
    compound = analyzer.polarity_scores(comment)["compound"]
    pos = analyzer.polarity_scores(comment)["pos"]
    neu = analyzer.polarity_scores(comment)["neu"]
    neg = analyzer.polarity_scores(comment)["neg"]
    
    scores["Compound"].append(compound)
    scores["Positive"].append(pos)
    scores["Negative"].append(neg)
    scores["Neutral"].append(neu)
sentiments_score = pd.DataFrame(scores)
content = pd.concat([content, sentiments_score], axis=1)

In [26]:
content.to_csv('sentiment_clean_data.csv', index=False)

In [27]:
content=pd.read_csv('sentiment_clean_data.csv', encoding='latin1')

In [28]:
content

,comment_id,self_text,created_time,score,subreddit,clean_text,Compound,Positive,Negative,Neutral
0,lfqhets,Have you ever read Hamas mission statement fro...,2024-07-31 00:00:00,1,IsraelPalestine,ever read hamas mission statement 1988 recentl...,-0.6705,0.158,0.232,0.611
1,lfqhbzr,You canÃ¢ÂÂt achieve zero casualties. But y...,2024-07-30 23:59:31,-1,IsraelPalestine,achieve zero casualty honest say side care kil...,-0.6486,0.244,0.332,0.424
2,lfqhb8g,You believe helping terrorists will serve your...,2024-07-30 23:59:23,6,IsraelPalestine,believe help terrorist serve interest,0.0000,0.460,0.379,0.161
3,lfqh7c9,"They'll commit the worst genocide ever, then t...",2024-07-30 23:58:42,1,IsraelPalestine,commit bad genocide ever terrorist run country...,-0.8779,0.077,0.396,0.526
4,lfqh6tr,"25,000 is a long shot. It took about 3000 Hama...",2024-07-30 23:58:37,1,IsraelPalestine,long shot take 3000 hamas militant kill less h...,-0.9925,0.107,0.255,0.638
...,...,...,...,...,...,...,...,...,...,...
709935,jz9k1yo,He is trying to stay calm for his daughter. Ã¢...,2023-09-05 17:56:26,2614,TerrifyingAsFuck,try stay calm daughter,0.3182,0.434,0.000,0.566
709936,jz9jfca,That wasn't a robbery. they didn't take anythi...,2023-09-05 17:52:37,297,TerrifyingAsFuck,robbery take anything sound like hit,0.3612,0.333,0.000,0.667
709937,jz9j8xh,Robbery that was a hit. That guy had planned o...,2023-09-05 17:51:31,47,TerrifyingAsFuck,robbery hit guy plan kill even move towards,-0.6908,0.000,0.402,0.598
709938,jz9j3lj,After this do they go home to their wife and k...,2023-09-05 17:50:37,1774,TerrifyingAsFuck,go home wife kid act normal like traumatized c...,-0.0516,0.176,0.190,0.634


In [29]:
content['Compound']

0        -0.6705
1        -0.6486
2         0.0000
3        -0.8779
4        -0.9925
           ...  
709935    0.3182
709936    0.3612
709937   -0.6908
709938   -0.0516
709939   -0.9531
Name: Compound, Length: 709940, dtype: float64

In [30]:
conditions = [
    (content['Compound'] <= -0.5),
    (content['Compound'] > -0.5) & (content['Compound'] < 0.5),
    (content['Compound'] > 0.5)
]
values = ['Negative', 'Neutral', 'Positive']
content['Category'] = np.select(conditions, values, default='Unknown')

In [31]:
content_filtered = content[content['Category'] != 'Unknown']


In [32]:
content_filtered.to_csv('content_filtered.csv', index=True)

In [33]:
content_filtered=pd.read_csv('content_filtered.csv', encoding='latin1', index_col=0)

MemoryError: Unable to allocate 32.5 MiB for an array with shape (6, 709913) and data type object

In [ ]:
content_filtered

,comment_id,self_text,created_time,score,subreddit,clean_text,Compound,Positive,Negative,Neutral,Category
0,lfqhets,Have you ever read Hamas mission statement fro...,2024-07-31 00:00:00,1,IsraelPalestine,ever read hamas mission statement 1988 recentl...,-0.6705,0.158,0.232,0.611,Negative
1,lfqhbzr,You canÃÂ¢ÃÂÃÂt achieve zero casualties. ...,2024-07-30 23:59:31,-1,IsraelPalestine,achieve zero casualty honest say side care kil...,-0.6486,0.244,0.332,0.424,Negative
2,lfqhb8g,You believe helping terrorists will serve your...,2024-07-30 23:59:23,6,IsraelPalestine,believe help terrorist serve interest,0.0000,0.460,0.379,0.161,Neutral
3,lfqh7c9,"They'll commit the worst genocide ever, then t...",2024-07-30 23:58:42,1,IsraelPalestine,commit bad genocide ever terrorist run country...,-0.8779,0.077,0.396,0.526,Negative
4,lfqh6tr,"25,000 is a long shot. It took about 3000 Hama...",2024-07-30 23:58:37,1,IsraelPalestine,long shot take 3000 hamas militant kill less h...,-0.9925,0.107,0.255,0.638,Negative
...,...,...,...,...,...,...,...,...,...,...,...
709935,jz9k1yo,He is trying to stay calm for his daughter. Ã...,2023-09-05 17:56:26,2614,TerrifyingAsFuck,try stay calm daughter,0.3182,0.434,0.000,0.566,Neutral
709936,jz9jfca,That wasn't a robbery. they didn't take anythi...,2023-09-05 17:52:37,297,TerrifyingAsFuck,robbery take anything sound like hit,0.3612,0.333,0.000,0.667,Neutral
709937,jz9j8xh,Robbery that was a hit. That guy had planned o...,2023-09-05 17:51:31,47,TerrifyingAsFuck,robbery hit guy plan kill even move towards,-0.6908,0.000,0.402,0.598,Negative
709938,jz9j3lj,After this do they go home to their wife and k...,2023-09-05 17:50:37,1774,TerrifyingAsFuck,go home wife kid act normal like traumatized c...,-0.0516,0.176,0.190,0.634,Neutral


In [ ]:
new_category_counts = content_filtered.groupby(['Category'])['Category'].count()
new_counts_df = pd.DataFrame(new_category_counts).rename(columns={"Category": "Counts"})
print(new_counts_df.index)

Index(['Negative', 'Neutral', 'Positive'], dtype='object', name='Category')


In [ ]:
new_counts_df['Percentage'] = (new_counts_df['Counts'] / new_counts_df['Counts'].sum()) * 100
print(new_counts_df.index)

Index(['Negative', 'Neutral', 'Positive'], dtype='object', name='Category')


In [ ]:
#ew_counts_df.to_csv('new_counts_df.csv', index=True)